In [1]:
import pandas as pd
import re

In [2]:
df_1 = pd.read_csv("tabla_grados.csv")

df_1['nombre_grado'] = df_1['nombre_grado'].ffill()

df_1 = df_1.drop(columns=['rama', 'area', 'descripcion', 'salidas'])

def limpiar(texto):
    if pd.isna(texto):
        return ""
    t = texto
    
    t = re.sub(r"©\s*Comunidad de Madrid.*?(?=$)", "", t)
    t = re.sub(r"Edición\s*:\s*\d{1,2}/\d{4}", "", t)
    t = re.sub(r"ISBN\s*:\s*\d{9,}", "", t)
    t = re.sub(r"Centro de Información y Asesoramiento Universitario.*?universidades", "", t, flags=re.I)
    t = re.sub(r"informacion\.universitaria@madrid\.org", "", t, flags=re.I)
    t = re.sub(r"c\.madrid/universidades", "", t, flags=re.I)
    t = re.sub(r"Publicado en España.*", "", t, flags=re.I)
    t = re.sub(r"Más información:.*?universidades", "", t, flags=re.I)
    t = re.sub(r"Ana Moralejo", "", t, flags=re.I)

    t = re.sub(r"\s+", " ", t).strip(" .,")

    return t

df_1['centros'] = df_1['centros'].apply(limpiar)
df_1['nombre_grado'] = df_1['nombre_grado'].apply(limpiar)

df_1.to_csv("grados_limpios.csv", index=False, encoding="utf-8")

In [3]:
df = pd.read_csv("grados_limpios.csv")

def limpiar_bloque(texto):
    if pd.isna(texto):
        return ""
    t = re.sub(r"\s+", " ", texto)
    t = re.sub(r"Direcciones de los centros.*?(?=Universidad|Centro|Escuela|$)", "", t, flags=re.I)
    t = re.sub(r"de Grado en [^U]+(?=Universidad|Centro|Escuela)", "", t)
    return t.strip()

def dividir_centros(texto):
    texto = limpiar_bloque(texto)
    if not texto:
        return []

    texto = re.sub(
        r"(?i)(C/|Avda\.?|Avenida|Paseo|Plaza|Camino|Carretera)[^\.]{0,60}Universidad",
        lambda m: m.group(0).replace("Universidad", "__UNI_CALLE__"),
        texto
    )

    pattern = r"(?=(?:Universidad\s+[^(]{0,100}\((?:Pública|Privada)))"

    partes = re.split(pattern, texto)

  
    partes_limpias = []
    for p in partes:
        p = p.replace("__UNI_CALLE__", "Universidad").strip(" .;:")
        if not p:
            continue
        if re.match(r"^(Escuela|Centro|Facultad|Instituto)\b", p, flags=re.I) and partes_limpias:
            partes_limpias[-1] += " " + p  # unir con la anterior
        else:
            partes_limpias.append(p)

    return partes_limpias


def extraer_info(bloque, grado):
    bloque = str(bloque).strip()
    if not bloque or re.search(r"(?i)Adscrito", bloque):
        return None

    # Buscar direcciones
    dir_pat = r"(?:C/|Avd\.?|Urb|Paseo|Pza|Camino|Crta)[^\n@]{10,180}?(?=(?:Tel|Tels|www|@|$))"
    direcciones = re.findall(dir_pat, bloque, flags=re.I)

    # Universidad y tipo (antes de la primera dirección)
    antes_dir = bloque.split(direcciones[0])[0] if direcciones else bloque
    u_match = re.search(r"^(Universidad|Centro|Escuela|Instituto)\s+([^()]+)\((Pública|Privada)", antes_dir)

    universidad = tipo = None
    texto_para_facultad = antes_dir  # Texto donde buscaremos la facultad

    if u_match:
        universidad = f"{u_match.group(1)} {u_match.group(2).strip()}"
        tipo = u_match.group(3)

        texto_para_facultad = antes_dir[u_match.end():]

    # Facultad o escuela
    f_match = re.search(r"(Facultad|Escuela|Centro)[^.:\n@]+", texto_para_facultad)
    facultad = f_match.group(0).strip() if f_match else None

    if facultad and re.search(r"(?i)\bC/\b", facultad):
        facultad = re.split(r"(?i)\bC/\b", facultad)[0].strip(" ,.;:")

    # Teléfonos, webs, emails
    telefonos = " / ".join(sorted(set(re.findall(r"\b\d{2,3}\s?\d{2,3}\s?\d{2}\s?\d{2}\b", bloque)))) or None
    webs = " | ".join(sorted(set(re.findall(r"(?:https?://|www\.)[^\s,;]+", bloque)))) or None
    emails = " | ".join(sorted(set(re.findall(r"[\w\.-]+@[\w\.-]+\.\w+", bloque)))) or None

    grado = str(grado) if not pd.isna(grado) else ""

    # Generar una fila por cada dirección detectada
    filas = []
    if direcciones:
        for d in direcciones:
            filas.append({
                "nombre_grado": grado.strip(),
                "universidad": universidad.strip() if universidad else None,
                "tipo": tipo.strip() if tipo else None,
                "facultad": facultad,
                "direccion": d.strip(" ,.;:"),
                "telefonos": telefonos,
                "webs": webs,
                "emails": emails
            })
    else:
        filas.append({
            "nombre_grado": grado.strip(),
            "universidad": universidad.strip() if universidad else None,
            "tipo": tipo.strip() if tipo else None,
            "facultad": facultad,
            "direccion": None,
            "telefonos": telefonos,
            "webs": webs,
            "emails": emails
        })

    return filas


filas = []
for _, fila in df.iterrows():
    centros = dividir_centros(fila.get("centros", ""))
    grado = fila.get("nombre_grado", "")
    for c in centros:
        info_list = extraer_info(c, grado)
        if info_list:
            filas.extend(info_list)


df_centros = pd.DataFrame(filas)
df_centros.drop_duplicates(subset=["nombre_grado", "universidad", "facultad", "direccion"], inplace=True)
df_centros.to_csv("centros_universitarios.csv", index=False, encoding="utf-8")
